In [1]:
if(!require(caret)) {install.packages("caret"); library(caret) }
if(!require(DescTools)) {install.packages("DescTools"); library(DescTools) }
if(!require(mice)) {install.packages("mice"); library(mice) }
if(!require(dplyr)) {install.packages("dplyr"); library(dplyr) }

Loading required package: caret

Loading required package: ggplot2

Loading required package: lattice

Loading required package: DescTools


Attaching package: ‘DescTools’


The following objects are masked from ‘package:caret’:

    MAE, RMSE


Loading required package: mice


Attaching package: ‘mice’


The following object is masked from ‘package:stats’:

    filter


The following objects are masked from ‘package:base’:

    cbind, rbind


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [37]:
load("processed.RData")
head(base)

,Age,Gender,BMI,isReadmission,UnitLengthStay,AdmissionSourceName,Frailty,DeathProbability,SofaScore,AdmissionMainDiagnosisName,⋯,FiO224h,HighestHeartRate24h,HighestRespiratoryRate24h,HighestTemperature24h,HighestLeukocyteCount24h,HighestCreatinine24h,Bilirubin24h,Lactate24h,Urea24h,IsMechanicalVentilation24h
,<dbl>,<fct>,<dbl>,<lgl>,<dbl>,<fct>,<lgl>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,41.85,M,NA,FALSE,39.16667,2,FALSE,0.000739,2,,⋯,0.21,100,20,37.5,20.21,86,8,0.8,6.6,FALSE
2,67.00,F,NA,FALSE,64.50000,3,NA,0.058088,4,,⋯,0.21,124,33,39.2,10.60,63,19,NA,4.8,FALSE
3,70.14,F,NA,FALSE,135.31667,2,FALSE,0.170892,2,,⋯,NA,118,30,36.7,17.50,74,46,1.2,7.7,FALSE
4,88.20,M,NA,FALSE,44.91667,1,NA,0.050472,1,,⋯,0.28,75,24,36.7,13.80,69,10,0.0,6.7,TRUE
5,51.20,M,40.2725,FALSE,112.56667,2,NA,0.001877,1,,⋯,NA,83,22,37.0,8.30,52,NA,NA,4.1,FALSE
6,65.80,M,NA,FALSE,137.91667,2,NA,0.079235,3,,⋯,0.36,178,34,37.1,4.88,60,8,2.3,7.9,FALSE


In [39]:
q = quantile(base$UnitLengthStay, prob=c(0.25, 0.5, 0.75))
q1 = as.numeric(q[1])
q3 = as.numeric(q[3])
outlier = q3 + 1.5*IQR(base$UnitLengthStay)
outlier


[1] 163.375

In [40]:
set.seed(998)
inTraining <- createDataPartition(base$UnitLengthStay,
                                  p = .8, list = FALSE)
training <- base[ inTraining,]
training$UnitLengthStay[training$UnitLengthStay > outlier] <- NA
testing  <- base[-inTraining,]
testing$UnitLengthStay[testing$UnitLengthStay > outlier] <- NA


In [41]:
nzv = nearZeroVar(training, saveMetrics = T, freqCut = 100/2)
nzv["Variaveis"] = row.names(nzv)
descritiva_nzv = nzv%>%
  filter(nzv==T)%>%
  select(Variaveis,freqRatio,percentUnique)
retirados_nzv = descritiva_nzv$Variaveis

In [35]:
training = training %>%
  select(.,-retirados_nzv)
testing = testing %>%
  select(.,-retirados_nzv)

In [36]:
# Identifying and Removing Correlated Predictors (for numeric features)
training_pre_numeric = training %>%
  select_if(., is.numeric)
training_pre_numeric$UnitLengthStay = NULL
descrCor <-  cor(training_pre_numeric, 
                 use="pairwise.complete.obs")
highlyCorDescr <- findCorrelation(descrCor, cutoff = .75)
retirados_cor = colnames(training_pre_numeric[,highlyCorDescr])
training_pre_numeric = 
  training_pre_numeric[,-highlyCorDescr]
   
testing_pre_numeric = testing %>%
  select_if(., is.numeric)
testing_pre_numeric$UnitLengthStay = NULL
testing_pre_numeric = 
  testing_pre_numeric[,-highlyCorDescr]

In [37]:
# Identifying and Removing Correlated Predictors (for categorical features)
training_pre_factor = training %>%
  select_if(., is.factor)
cramer_tab = PairApply(training_pre_factor,
                       CramerV, symmetric = TRUE)
cramer_tab[which(is.na(cramer_tab[,])==T)] = 0
highlyCorCateg <- findCorrelation(cramer_tab, cutoff = 0.5)
retirados_categ = colnames(training_pre_factor[,highlyCorCateg])
training_pre_factor = training_pre_factor %>%
  select(.,-retirados_categ)

testing_pre_factor = testing %>%
  select_if(., is.factor)
testing_pre_factor = testing_pre_factor %>%
  select(.,-retirados_categ)

training = cbind(training_pre_numeric,training_pre_factor, training$UnitLengthStay)
training$UnitLengthStay = training$`training$UnitLengthStay`
training$`training$UnitLengthStay` = NULL

testing = cbind(testing_pre_numeric,testing_pre_factor, testing$UnitLengthStay)
testing$UnitLengthStay = testing$`testing$UnitLengthStay`
testing$`testing$UnitLengthStay` = NULL


In [38]:
#MICE Imputation
training_imp = training
testing_imp = testing

  #training
set.seed(100)
predictormatrix = quickpred(training_imp,
                          include = c("UnitLengthStay"),
                          exclude = NULL,
                          mincor = 0.3)
imp_gen = mice(data = training_imp,
               predictorMatrix = predictormatrix,
               m=1,
               maxit = 5,
               diagnostics=TRUE)

imp_data = mice::complete(imp_gen,1)
training_imp = imp_data
summary(training_imp)



 iter imp variable
  1   1  BMI  DeathProbability  LowestSystolicBloodPressure24h  LowestDiastolicBloodPressure24h  LowestGlasgowComaScale24h  LowestPlateletsCount24h  pH24h  PaO24h  PaCO21h  FiO224h  HighestHeartRate24h  HighestRespiratoryRate24h  HighestTemperature24h  HighestLeukocyteCount24h  HighestCreatinine24h  Bilirubin24h  Lactate24h  Urea24h  Gender  AdmissionSourceName  IsChfNyhaClass4  IsCrfDialysis  IsSolidTumorMetastatic  IsSevereCopd  IsDiabetes  IsNonInvasiveVentilation  IsVasopressors  IsRenalReplacementTherapy
  2   1  BMI  DeathProbability  LowestSystolicBloodPressure24h  LowestDiastolicBloodPressure24h  LowestGlasgowComaScale24h  LowestPlateletsCount24h  pH24h  PaO24h  PaCO21h  FiO224h  HighestHeartRate24h  HighestRespiratoryRate24h  HighestTemperature24h  HighestLeukocyteCount24h  HighestCreatinine24h  Bilirubin24h  Lactate24h  Urea24h  Gender  AdmissionSourceName  IsChfNyhaClass4  IsCrfDialysis  IsSolidTumorMetastatic  IsSevereCopd  IsDiabetes  IsNonInvasiveVenti

      Age              BMI            DeathProbability     SofaScore     
 Min.   : 18.00   Min.   :    0.266   Min.   :0.000000   Min.   : 0.000  
 1st Qu.: 51.90   1st Qu.:   24.032   1st Qu.:0.004477   1st Qu.: 1.000  
 Median : 65.80   Median :   27.757   Median :0.014799   Median : 3.000  
 Mean   : 62.56   Mean   :   31.038   Mean   :0.081889   Mean   : 3.535  
 3rd Qu.: 75.51   3rd Qu.:   32.873   3rd Qu.:0.066915   3rd Qu.: 5.000  
 Max.   :109.97   Max.   :12430.000   Max.   :0.999625   Max.   :22.000  
                                                                         
 LowestSystolicBloodPressure24h LowestDiastolicBloodPressure24h
 Min.   :  1.00                 Min.   :  1.00                 
 1st Qu.: 88.00                 1st Qu.: 46.00                 
 Median : 98.00                 Median : 52.00                 
 Mean   : 98.57                 Mean   : 53.81                 
 3rd Qu.:110.00                 3rd Qu.: 60.00                 
 Max.   :248.00         

In [39]:
  #testing
set.seed(100)
predictormatrix = quickpred(testing_imp,
                            include = c("UnitLengthStay"),
                            exclude = NULL,
                            mincor = 0.3)
imp_gen_test = mice(data = testing_imp,
               predictorMatrix = predictormatrix,
               m=1,
               maxit = 5,
               diagnostics=TRUE)
imp_data_test = mice::complete(imp_gen_test,1)
testing_imp = imp_data_test
summary(testing_imp)


 iter imp variable
  1   1  BMI  DeathProbability  LowestSystolicBloodPressure24h  LowestDiastolicBloodPressure24h  LowestGlasgowComaScale24h  LowestPlateletsCount24h  pH24h  PaO24h  PaCO21h  FiO224h  HighestHeartRate24h  HighestRespiratoryRate24h  HighestTemperature24h  HighestLeukocyteCount24h  HighestCreatinine24h  Bilirubin24h  Lactate24h  Urea24h  Gender  AdmissionSourceName  IsChfNyhaClass4  IsCrfDialysis  IsSolidTumorMetastatic  IsSevereCopd  IsDiabetes  IsNonInvasiveVentilation  IsVasopressors  IsRenalReplacementTherapy
  2   1  BMI  DeathProbability  LowestSystolicBloodPressure24h  LowestDiastolicBloodPressure24h  LowestGlasgowComaScale24h  LowestPlateletsCount24h  pH24h  PaO24h  PaCO21h  FiO224h  HighestHeartRate24h  HighestRespiratoryRate24h  HighestTemperature24h  HighestLeukocyteCount24h  HighestCreatinine24h  Bilirubin24h  Lactate24h  Urea24h  Gender  AdmissionSourceName  IsChfNyhaClass4  IsCrfDialysis  IsSolidTumorMetastatic  IsSevereCopd  IsDiabetes  IsNonInvasiveVenti

      Age              BMI           DeathProbability     SofaScore     
 Min.   : 18.00   Min.   :   0.283   Min.   :0.000000   Min.   : 0.000  
 1st Qu.: 51.92   1st Qu.:  24.049   1st Qu.:0.004508   1st Qu.: 1.000  
 Median : 65.80   Median :  27.778   Median :0.014928   Median : 3.000  
 Mean   : 62.62   Mean   :  31.172   Mean   :0.082188   Mean   : 3.533  
 3rd Qu.: 75.58   3rd Qu.:  32.966   3rd Qu.:0.067086   3rd Qu.: 5.000  
 Max.   :109.07   Max.   :9173.554   Max.   :0.998245   Max.   :21.000  
                                                                        
 LowestSystolicBloodPressure24h LowestDiastolicBloodPressure24h
 Min.   :  1.00                 Min.   :  1.00                 
 1st Qu.: 88.00                 1st Qu.: 46.00                 
 Median : 98.00                 Median : 52.00                 
 Mean   : 98.54                 Mean   : 53.75                 
 3rd Qu.:110.00                 3rd Qu.: 60.00                 
 Max.   :227.00                 

In [40]:
#Final preprocessed dataset
write.csv(training, 'training.csv', row.names = FALSE)
write.csv(testing, 'testing.csv', row.names = FALSE)